# Gradient Descent

- Initialize $\mathbf{w}(0)$
- For $t=0,1,2, \cdots \quad$ [to termination]
$$
\mathbf{w}(t+1)=\mathbf{w}(t)-\eta \nabla E_{\text {in }}(\mathbf{w}(t))
$$
- Return final w

With **Stochastic** gradient descent, we pick one $(x_n, y_n)$ at a time, applying Gradient Descent to $e(h(x_n), y_n)$.

Rule of thumb: $\eta = .1$ works.

# How the Network Operates

$$
w_{i j}^{(l)} \quad \begin{cases}1 \leq l \leq L & \text { layers } \\ 0 \leq i \leq d^{(l-1)} & \text { inputs } \\ 1 \leq j \leq d^{(l)} & \text { outputs }\end{cases}
$$

$$
\theta(s)=\tanh (s)=\frac{e^s-e^{-s}}{e^s+e^{-s}}
$$

$$
x_j^{(l)}=\theta\left(s_j^{(l)}\right)=\theta\left(\sum_{i=0}^{d^{(l-1)}} w_{i j}^{(l)} x_i^{(l-1)}\right)
$$

$$
\text { Apply } \mathbf{x} \text { to } x_1^{(0)} \cdots x_{d^{(0)}}^{(0)} \rightarrow \rightarrow x_1^{(L)}=h(\mathbf{x})
$$



# Applying SGD

Error on example $\left(\mathbf{x}_n, y_n\right)$ is
$$
\mathrm{e}\left(h\left(\mathbf{x}_n\right), y_n\right)=\mathrm{e}(\mathrm{w})
$$

To implement SGD, we need the gradient: 

$$
\nabla \mathrm{e}(\mathbf{w}): \frac{\partial \mathrm{e}(\mathrm{w})}{\partial w_{i j}^{(l)}} \text { for all } i, j, l
$$

# Computing $\frac{\partial \mathrm{e}(\mathbf{w})}{\partial w_{i j}^{(l)}}$

A trick for efficient computation:
$$
\frac{\partial \mathrm{e}(\mathbf{w})}{\partial w_{i j}^{(l)}}=\frac{\partial \mathrm{e}(\mathbf{w})}{\partial s_j^{(l)}} \times \frac{\partial s_j^{(l)}}{\partial w_{i j}^{(l)}}
$$

$$
\text { We have } \frac{\partial s_j^{(l)}}{\partial w_{i j}^{(l)}}=x_i^{(l-1)} \quad \text { We only need: } \frac{\partial \mathrm{e}(\mathrm{w})}{\partial s_j^{(l)}}=\delta_j^{(l)}
$$

# $\delta$ for the final layer

For the final layer $l=L$ and $j=1$
$$
\begin{aligned}
& \delta_1^{(L)}=\frac{\partial \mathrm{e}(\mathbf{w})}{\partial s_1^{(L)}} \\
& = \frac{\partial(\theta(s_1^{(L)}) - y_n)^2}{\partial s_1^{(L)}} \\
& = 2(\theta(s_1^{(L)}) - y_n)(\theta^{\prime}(s_1^{(L)})) \\
& = 2(x_1^{(L)} - y_n)(\theta^{\prime}(s_1^{(L)})) \\
& = 2(x_1^{(L)} - y_n)(1 - \theta^2(s_1^{(L)})) \\
& = 2(x_1^{(L)} - y_n)(1 - (x_1^{(L)})^2)
\end{aligned}
$$

# Back propagation of $\delta$

$$
\begin{aligned}
\delta_i^{(l-1)} & =\frac{\partial \mathrm{e}(\mathbf{w})}{\partial s_i^{(l-1)}} \\
& =\sum_{j=1}^{d^{(l)}} \frac{\partial \mathrm{e}(\mathbf{w})}{\partial s_j^{(l)}} \times \frac{\partial s_j^{(l)}}{\partial x_i^{(l-1)}} \times \frac{\partial x_i^{(l-1)}}{\partial s_i^{(l-1)}} \\
& =\sum_{j=1}^{d^{(l)}} \delta_j^{(l)} \times w_{i j}^{(l)} \times \theta^{\prime}\left(s_i^{(l-1)}\right) \\
\delta_i^{(l-1)} & =\left(1-\left(x_i^{(l-1)}\right)^2\right) \sum_{j=1}^{d^{(l)}} w_{i j}^{(l)} \delta_j^{(l)}
\end{aligned}
$$

![image.png](../images/network-architecture.png)

In [153]:
import numpy as np

# Stochastic gradient descent with multilayer 2D perceptrons.  Takes an array of 2d points as inputs, a target function, and a value for eta, and runs sgd, ultimately returning the weights.
DEFAULT_ETA = .1

# https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/
    # " an MLP with two hidden layers is sufficient for creating classification regions of any desired shape. This is instructive, although it should be noted that no indication of how many nodes to use in each layer"
LAYER_STRUCTURE = [2, 3, 2, 1] # Two hidden layers, with 3 neurons and 2 neurons respectively.  Note that these counts do not include the bias neurons, which are indexed at 0.  See the structure in the image above.

ACTIVATION_FUNCTION = np.tanh

# A single final output is assumed.

#TODO: this would be better as a Class, due to var sharing
def sgd(inputs, outputs, eta = DEFAULT_ETA, layer_structure=LAYER_STRUCTURE):
    total_layers = len(LAYER_STRUCTURE) # Input layer + hidden layers + output layer
    # Initialize all neurons x with empty values
    x = np.empty(total_layers, dtype=object)
    for l, neuron_count in enumerate(LAYER_STRUCTURE):
        x[l] = np.empty(neuron_count + 1)
        if l < (len(LAYER_STRUCTURE) - 1):
            x[l][0] = 1 # 1 for bias
        else:
            x[l][0] = None # No bias on the final layer.  A placeholder to get 1-based indexing of real neurons.

    # Initialize all weights w_{ij}^(l) with Xavier weight initialization..
    # Note that our weights include biases.
    weights = np.empty(total_layers, dtype=object)

    for l, neuron_count in enumerate(LAYER_STRUCTURE):
        if l == 0:
            prev_neuron_count = neuron_count
            continue # skip input layer
        weights[l] = np.empty((prev_neuron_count + 1, neuron_count + 1)) 
        weights[l][0,:] = 0.0 # initialize biases to 0
        for i in range(1, prev_neuron_count + 1):
            for j in range(1, neuron_count + 1):
                weights[l][i][j] = xavier_weight(prev_neuron_count, neuron_count)
        prev_neuron_count = neuron_count

    # TODO: Proly worth reporting e_in/e_out on our bogus starting weights.

    # Initialize the array of signal derivatives
    d = np.empty(total_layers, dtype=object)
    for l, neuron_count in enumerate(LAYER_STRUCTURE):
        if l == 0:
            continue # skip input layer
        d[l] = np.empty(neuron_count + 1)

    epoch_count = 0 
    #TODO: ultimately, will want something like out of sample validation to end the loop
    while (epoch_count < 100):
        # Generate a random permutation for how we pick the inputs
        input_perm = np.random.permutation(len(inputs))
        for input_id in input_perm:
            input = inputs[input_id]

            #TODO: extract common feedforward code
            # Forward: Compute all x_j^(l)
            # First we set layer 1 values
            x[0] = input

            # Now we forward propagate x
            # NOTE: this could be vectorized (but would have to address the empty value in weights)
            for l in range(1, total_layers):
                for j in range(1, len(x[l])):
                    x[l][j] = ACTIVATION_FUNCTION(x[l - 1].dot(weights[l][:,j]))

            # Backward: Compute all d_j^(l)
            # Begin by computing d_1^(L) for the final layer
            d[-1][1] = 2 * (x[-1][1] - outputs[input_id]) * (1 - (x[-1][1] ** 2))

            # Now we loop through and calculate the previous d's backward..
            for l in reversed(range(1,total_layers - 1)):
                for i in range(1, LAYER_STRUCTURE[l]):
                    d[l][i] = (1 - ((x[l][i]) ** 2)) * weights[l+1][i][1:].dot(d[l + 1][1:])

            # Update the weights: w_{ij}^(l) = w_{ij}^(l) - (eta * x_i^(l-1) * d_j^(l))
            # We'd never have j= 1 as an output
            
            for l in range(1, total_layers):
                for i in range(0, LAYER_STRUCTURE[l-1]):
                    for j in range(1, LAYER_STRUCTURE[l]):
                        weights[l][i][j] = weights[l][i][j] - (eta * x[l-1][i] * d[l][j])
            
        # Report E_in and e_out for the epoch.  Report current weights too.
        print("Epoch " + str(epoch_count))
        total_e_in = 0
        # First E_in
        for input_id, input in enumerate(inputs):
            #TODO: extract common feedforward code
            x[0] = input

            # Now we forward propagate x
            # NOTE: this could be vectorized (but would have to address the empty value in weights)
            for l in range(1, total_layers):
                for j in range(1, len(x[l])):
                    x[l][j] = ACTIVATION_FUNCTION(x[l - 1].dot(weights[l][:,j]))
            
            h = x[-1][1]
            # TODO: extract common err code
            e_in = (h - outputs[input_id]) ** 2
            total_e_in += e_in
        
        avg_e_in = total_e_in / len(inputs)
        print("Avg E_in: " + str(avg_e_in))
        
        # Now for e_out
        total_e_out = 0
        # TODO: extract generation of data code
        n = 1000 # 1000 training inputs, 1000 testing seems good?
        # QSTN: What's a good ratio between training N and testing N?
        testing_inputs = rng.uniform(-1,1,(n, 2))
        testing_inputs = np.hstack((np.ones((n,1)), testing_inputs)) # Set x_0 = 1 for all xs
        testing_outputs = np.sign(.5 - (np.square(testing_inputs[:,1]) + np.square(testing_inputs[:,2])))

        for testing_input_id, testing_input in enumerate(testing_inputs):
            #TODO: extract common feedforward code
            x[0] = testing_input

            # Now we forward propagate x
            # NOTE: this could be vectorized (but would have to address the empty value in weights)
            for l in range(1, total_layers):
                for j in range(1, len(x[l])):
                    x[l][j] = ACTIVATION_FUNCTION(x[l - 1].dot(weights[l][:,j]))
            
            h = x[-1][1]
            # TODO: extract common err code
            e_out = (h - testing_outputs[testing_input_id]) ** 2
            total_e_out += e_out
        
        avg_e_out = total_e_out / len(testing_inputs)
        print("Avg E_out: " + str(avg_e_out))        

        epoch_count += 1

        # Break the loop if it's time to stop [ might be done through an e_out target]
        # QSTN: when is the right time to stop in SGD?  how do we identify that?
            # Note, overfitting is a concern here https://stats.stackexchange.com/questions/433187/stopping-criteria-for-stochastic-gradient-descent , validation is part of how it's addressed.  Not sure how much I want to postpone my v1 over this..

    # Return the weights

# looking like https://machinelearningmastery.com/weight-initialization-for-deep-learning-neural-networks/, Xavier weight initialization is a good approach. 
# Note: biases don't count towards count, so subtract them off before passing in
def xavier_weight(prev_neuron_count, neuron_count):
    return np.random.uniform(-np.sqrt(6)/np.sqrt(prev_neuron_count + neuron_count),np.sqrt(6)/np.sqrt(prev_neuron_count + neuron_count))


# Our inputs
n = 10
rng = np.random.default_rng()
inputs = rng.uniform(-1,1,(n, 2))
inputs = np.hstack((np.ones((n,1)), inputs)) # Set x_0 = 1 for all xs

# Circle function (our target function):
# If x_1^2 + x_2^2 < .5, then we are inside our circle (return +1) (which has radius of sqrt(.5) =~ .707).  else return -1.
outputs = np.sign(.5 - (np.square(inputs[:,1]) + np.square(inputs[:,2])))

# Our target function
target_function = None

print(sgd(inputs, outputs))


Epoch 0
Avg E_in: 1.080632067283989
Avg E_out: 0.9682425514585163
Epoch 1
Avg E_in: 0.9981968364142746
Avg E_out: 1.0272051574753187
Epoch 2
Avg E_in: 1.0440719140124097
Avg E_out: 1.141552601276721
Epoch 3
Avg E_in: 0.9913724880066347
Avg E_out: 1.019369097188882
Epoch 4
Avg E_in: 1.0202011724374942
Avg E_out: 1.0762673740368058
Epoch 5
Avg E_in: 0.99933942293521
Avg E_out: 1.049470956543511
Epoch 6
Avg E_in: 1.014020858676124
Avg E_out: 1.0695361150633313
Epoch 7
Avg E_in: 1.0136890518107353
Avg E_out: 1.073089136713895
Epoch 8
Avg E_in: 1.023961833025377
Avg E_out: 1.0801484290432295
Epoch 9
Avg E_in: 1.0444126224054644
Avg E_out: 1.1189418737045982
Epoch 10
Avg E_in: 1.012502800449149
Avg E_out: 1.0697946275784729
Epoch 11
Avg E_in: 1.0471034943302457
Avg E_out: 1.1204552750738246
Epoch 12
Avg E_in: 1.0042761199094374
Avg E_out: 1.051956861675655
Epoch 13
Avg E_in: 1.0077170833318354
Avg E_out: 1.0124062082910947
Epoch 14
Avg E_in: 1.008597524180453
Avg E_out: 1.0526005480463785
Ep